# Download data

The data in this example is a 64 channel recording using a cambridge neurotech ASSY-236-H5 probe. 

The probe file must be updated if any new recordings are used. 

In [16]:
# !gdown --id 1-jyxiDTD2FSLyB_CDGSyxdrWTm0POMv4
# !unzip /kaggle/working/cambridgeneurotech_openephys_recording.zip -d /kaggle/working/

from pathlib import Path
import subprocess

# Define base folder dynamically
base_folder = Path.cwd()
zip_path = base_folder / "cambridgeneurotech_openephys_recording.zip"
extract_to = base_folder / "cambridgeneurotech_openephys_recording"

# 1. Download the file from Google Drive
subprocess.run([
    "gdown", "--id", "1-jyxiDTD2FSLyB_CDGSyxdrWTm0POMv4",
    "-O", str(zip_path)
], check=True)

# 2. Unzip into a subfolder
subprocess.run([
    "unzip", str(zip_path), "-d", str(extract_to)
], check=True)

/home/ubuntu/miniconda3/envs/sorter/lib/python3.11/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-jyxiDTD2FSLyB_CDGSyxdrWTm0POMv4
From (redirected): https://drive.google.com/uc?id=1-jyxiDTD2FSLyB_CDGSyxdrWTm0POMv4&confirm=t&uuid=2155882b-51bc-4fb6-8b7c-b75ac2eebbfa
To: /home/ubuntu/codespace/sorter-script/cambridgeneurotech_openephys_recording.zip
100%|██████████| 939M/939M [00:10<00:00, 87.7MB/s] 


Archive:  /home/ubuntu/codespace/sorter-script/cambridgeneurotech_openephys_recording.zip


replace /home/ubuntu/codespace/sorter-script/cambridgeneurotech_openephys_recording/openephys_recording/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

KeyboardInterrupt: 

# Sort the data using Kilosort4

The data is wrapped in spike interface classes, which allow easy use of sorters, and also provide a curation GUI. 

Running this cell will detect, sort, and cluster the data. 

In [6]:
from spikeinterface.sorters import run_sorter
import spikeinterface.full as si
import probeinterface as pi
import scipy
from pathlib import Path
import numpy as np

# --- Load Open Ephys recording ---
base_folder = Path.cwd()
oe_folder = base_folder / 'cambridgeneurotech_openephys_recording/openephys_recording/2023-08-23_15-56-05'
rec = si.read_openephys(oe_folder, stream_id="0")

# --- Load probe and MAP its contacts to device channel indices ---
probe = pi.get_probe('cambridgeneurotech', 'ASSY-236-H5')

n_rec = rec.get_num_channels()
n_probe = probe.get_contact_count()

if n_probe != n_rec:
    raise ValueError(f"Probe contacts ({n_probe}) != recording channels ({n_rec}). "
                     f"Pick the correct probe variant or subset/remap accordingly.")

# Map 1:1 in current channel order
dev_inds = rec.ids_to_indices(rec.channel_ids)  # typically 0..N-1 in current order
probe.set_device_channel_indices(dev_inds)

# (optional) sanity check locations exist on the probe
assert probe.contact_positions is not None and probe.contact_positions.shape[0] == n_probe

# --- Attach probe (now it has device_channel_indices) ---
rec = rec.set_probe(probe, in_place=False)

# Quick check
print("Channel locations:", rec.get_channel_locations().shape)  # (N, 2)

# --- Run Kilosort4 ---
sorting_KS4 = run_sorter(
    sorter_name="kilosort4",
    recording=rec,
    folder="kilosort4_output",
    verbose=True,
)


Channel locations: (64, 2)


write_binary_recording (no parallelization): 100%|██████████| 300/300 [00:07<00:00, 38.17it/s]
kilosort.run_kilosort:  
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: N samples: 9000000
kilosort.run_kilosort: N seconds: 300.0
kilosort.run_kilosort: N batches: 150
kilosort.run_kilosort: Preprocessing filters computed in 0.50s; total 0.50s
kilosort.run_kilosort:  
kilosort.run_kilosort: Resource usage after preprocessing
kilosort.run_kilosort: ********************************************************
kilosort.run_kilosort: CPU usage:     4.70 %
kilosort.run_kilosort: Mem used:      4.70 %     |       2.94 GB
kilosort.run_kilosort: Mem avail:    59.16 / 62.10 GB
kilosort.run_kilosort: ------------------------------------------------------
kilosort.run_kilosort: GPU usage:    `conda install pynvml` for GPU usage
kilosort.run_kilosort: GPU memory:    3.10 %     |      0.45   /    14.56 GB
kiloso

kilosort4 run time 165.49s


# Create the sorting analyzer object

This object will contain the data used in the web interface

In [ ]:
import spikeinterface.full as si

base_folder = Path.cwd()
oe_folder = base_folder / "cambridgeneurotech_openephys_recording/openephys_recording/2023-08-23_15-56-05"
# sa_folder = base_folder / "sorting_analyzer"

recording = si.read_openephys(oe_folder, stream_id="0")
recording = recording.set_probe(probe, in_place=False)
recording_filtered = si.bandpass_filter(recording)


job_kwargs = dict(n_jobs=-1, progress_bar=True, chunk_duration="1s")

# make the SortingAnalyzer with necessary and some optional extensions
sorting_analyzer = si.create_sorting_analyzer(sorting_KS4, recording_filtered
                                              format="binary_folder", folder="sa_folder",
                                              **job_kwargs)
sorting_analyzer.compute("random_spikes", method="uniform", max_spikes_per_unit=500)
sorting_analyzer.compute("waveforms", **job_kwargs)
sorting_analyzer.compute("templates", **job_kwargs)
sorting_analyzer.compute("noise_levels")
sorting_analyzer.compute("unit_locations", method="monopolar_triangulation")
sorting_analyzer.compute("isi_histograms")
sorting_analyzer.compute("correlograms", window_ms=100, bin_ms=5.)
sorting_analyzer.compute("principal_components", n_components=3, mode='by_channel_global', whiten=True, **job_kwargs)
sorting_analyzer.compute("quality_metrics", metric_names=["snr", "firing_rate"])
sorting_analyzer.compute("template_similarity")
sorting_analyzer.compute("spike_amplitudes", **job_kwargs)


estimate_sparsity (workers: 16 processes): 100%|██████████| 300/300 [00:02<00:00, 147.75it/s]
/home/ubuntu/miniconda3/envs/sorter/lib/python3.11/site-packages/spikeinterface/core/basesorting.py:380: UserWarning: The registered recording will not be persistent on disk, but only available in memory
  warnings.warn("The registered recording will not be persistent on disk, but only available in memory")
spike_amplitudes (workers: 16 processes): 100%|██████████| 300/300 [00:02<00:00, 126.22it/s]


# Launch the web GUI

the spikes can be labled as good / bad / MUA, and merged

In [ ]:
from spikeinterface_gui import run_mainwindow
# reload the SortingAnalyzer
sorting_analyzer = sorting_analyzer
# open and run the Qt app
# open and run the Web app
run_mainwindow(sorting_analyzer, mode="web", curation=True)

/home/ubuntu/miniconda3/envs/sorter/lib/python3.11/site-packages/spikeinterface_gui/unitlist.py:436: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension("tabulator")


/home/ubuntu/miniconda3/envs/sorter/lib/python3.11/site-packages/spikeinterface_gui/mergeview.py:282: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension("tabulator")


/home/ubuntu/miniconda3/envs/sorter/lib/python3.11/site-packages/spikeinterface_gui/curationview.py:216: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension("tabulator")


/home/ubuntu/miniconda3/envs/sorter/lib/python3.11/site-packages/spikeinterface_gui/backend_panel.py:253: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension("gridstack")


Found available port: 33383
Launching server at http://localhost:33383


Merge not possible, some units are already deleted or in a merge group
